<a href="https://colab.research.google.com/github/bernardlawes/PyTorch-DeepLearning/blob/main/CNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as f
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Convert MNIST Image Files into a Tensor of 4 Dimensions (# of images, Height, Width, Color Channel)
transform = transforms.ToTensor()
# Train Data - note that datasets is the container that grants us access to MNIST and other datasets available through Torch
train_data = datasets.MNIST(root='/cnn_data', train=True, download=True, transform=transform)
# Test Data
test_data = datasets.MNIST(root='/cnn_data', train=False, download=True, transform=transform)

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
# Create a small batch size for images ... let's say 10
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

In [ ]:
# Define our CNN Model - we won't build CNN model here..  we will go about it step by step for this example
# Descrbe Convolutional layer and what it is doing (2 convolutional layers)
# this is just an example to illustrate how it all works

conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3, stride=1)   # 6 output channels represents 6 filters
conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3, stride=1)  # 6 outputs will become 6 inputs for this layer

In [ ]:
class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3, stride=1)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3, stride=1)
    # Fully Connected Layers
    self.fc1 = nn.Linear(in_features=400, out_features=120) # 5*5*16 - the output fom the extraction
    self.fc2 = nn.Linear(in_features=120, out_features=84)
    self.fc3 = nn.Linear(in_features=84, out_features=10)

  # Note the lowercase X so it doesn't confuse with the capital X used in the step by step section of this colab
  def forward(self, x):
    x = f.relu(self.conv1(x))
    x = f.max_pool2d(x, 2, 2) # 2x2 kernal and stride 2
    # Second Pass
    x = f.relu(self.conv2(x))
    x = f.max_pool2d(x, 2, 2) # 2x2 kernal and stride 2

    # Re-view to flatten it out
    x = x.view(-1, 400)  # negative so that we can vary the batch size

    # Fully Connected Layers
    x = f.relu(self.fc1(x))
    x = f.relu(self.fc2(x))
    x = self.fc3(x)

    return f.log_softmax(x, dim=1)

In [ ]:
# Create an Instance of our Model
seed = 41
torch.manual_seed(seed)
model = ConvolutionalNetwork()
model

In [ ]:
# Loss function optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #smaller the learning rate - the longer it going to train

In [ ]:
import time # so that we can keep track of time

start_time = time.time()

# Create variables to track things
epochs = 5
train_losses = [] # empty python list
test_losses = []
train_correct = []
test_correct = []

# For Loop of Epochs
for i in range(epochs):
  trn_corr = 0 # train correct
  tst_corr = 0 # test correct

  # Train

  for b, (X_train, y_train) in enumerate(train_loader):
    b+=1 # start our batches at 1

    y_pred = model(X_train) # get predcited values from traiing set.  Not flattened, it is 2D
    loss = criterion(y_pred, y_train) # how off are we?  Compare the predictions to the correct answers in y_train

    predicted = torch.max(y_pred.data,1)[1] # add up the number of correct preditions. Indexed off of first point
    batch_corr = (predicted == y_train).sum() # how mahy we got correct from this batch. True = 1. False = 0.
    trn_corr += batch_corr # keep track as we go along in training

    # Update our parameters
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print out our results
    if b%600 == 0:
      print(f'Epoch: {i} | Batch: {b} | Loss: {loss.item()}')

  train_losses.append(loss)
  train_correct.append(trn_corr)

  # Test
  with torch.no_grad(): # No gradient so that we don't update our weights and biases during test iterations
    for b, (X_test, y_test) in enumerate(test_loader): # test_loader instead of train_loader
      y_val = model(X_test) #
      predicted = torch.max(y_val.data, 1)[1] # adding up correct preditions
      tst_corr += (predicted == y_test).sum() # True = 1, False = 0, then sum

  loss = criterion(y_val, y_test)
  test_losses.append(loss)
  test_correct.append(tst_corr)

current_time = time.time()
total_time = current_time - start_time

print(f'Training Tool: {total_time} minutes |')

In [ ]:
# Graph the loss at epoch

train_losses = [t.detach().numpy() for t in train_losses]
plt.plot(train_losses, label='Training Loss')
plt.plot(test_losses, label='Testing Loss')
plt.title('Training and Testing Loss')
plt.legend()

In [ ]:
# Graph the accuracy at the end of each epoch

plt.plot([i/600 for i in train_correct], label='Training Accuracy')
plt.plot([i/100 for i in test_correct], label='Testing Accuracy')
plt.title('Training and Testing Accuracy')
plt.legend()

In [ ]:
test_load_everything = DataLoader(test_data, batch_size=10000, shuffle=False)

In [ ]:
with torch.no_grad():
  correct = 0
  for X_test, y_test in test_load_everything:
    y_val = model(X_test)
    predicted = torch.max(y_val, 1)[1]
    correct += (predicted == y_test).sum()

In [ ]:
# Test for Correct

correct.item()

In [ ]:
# Percentage Correct
100*correct.item()/10000

# Run Inference through the Model


In [ ]:
# Grab an Image Tensor - note that it includes the Label at the very end
test_data[4143] # Tensor with an image in it

In [ ]:
# Grab just the data, because we don't want the label
test_data[4143][0]

In [ ]:
#Reshape the data into a 28x28 image
test_data[4143][0].reshape(28,28)

In [ ]:
plt.imshow(test_data[4143][0].reshape(28,28), cmap='gray')

In [ ]:
plt.imshow(test_data[4143][0].reshape(28,28))

In [ ]:
model.eval()
with torch.no_grad():
  new_prediction = model(test_data[4143][0].reshape(1,1,28,28)) # batch of size 1, 1 color channel, 28x28 image

In [ ]:
# print out result
new_prediction

In [ ]:
# get highest prediction value
new_prediction.argmax()